In [55]:
spark.sql(
    """
    Drop DATABASE PL7G1_GOLD CASCADE
    """
)

DataFrame[]

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
# create silver database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS PL7G1_GOLD LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/'
    """
)

DataFrame[]

In [4]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.InstalacoesCounty
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.InstalacoesCounty (
        Year INT,
        Borough VARCHAR(50),
        Facility_Name VARCHAR(100),
        SUM_VOC Double,
        AVG_VOC Double,
        MAX_VOC Double,
        MIN_VOC Double,
        SUM_NOx Double,
        AVG_NOx Double,
        MAX_NOx Double,
        MIN_NOx Double,
        SUM_CO Double,
        AVG_CO Double,
        MAX_CO Double,
        MIN_CO Double,
        SUM_CO2 Double,
        AVG_CO2 Double,
        MAX_CO2 Double,
        MIN_CO2 Double,
        SUM_Particulates Double,
        AVG_Particulates Double,
        MAX_Particulates Double,
        MIN_Particulates Double,
        SUM_PM10 Double,
        AVG_PM10 Double,
        MAX_PM10 Double,
        MIN_PM10 Double,
        SUM_PM2_5 Double,
        AVG_PM2_5 Double,
        MAX_PM2_5 Double,
        MIN_PM2_5 Double,
        SUM_HAPS Double,
        AVG_HAPS Double,
        MAX_HAPS Double,
        MIN_HAPS Double,
        SUM_SO2 Double,
        AVG_SO2 Double,
        MAX_SO2 Double,
        MIN_SO2 Double,
       longitude VARCHAR(20),
       latitude VARCHAR(20)
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/InstalacoesCounty'
    """
)

DataFrame[]

In [6]:
from pyspark.sql.functions import substring, avg, sum, max, min
import pyspark.sql.functions as sf

# read data from the silver tables
instalacoes = spark.table("PL7G1_SILVER.instalacoes")

instalacoes = instalacoes \
           .groupBy("Year", "Borough", "Facility_Name", "latitude", "longitude") \
           .agg(sum(instalacoes.VOC).alias("SUM_VOC"),
                avg(instalacoes.VOC).alias("AVG_VOC"),
                max(instalacoes.VOC).alias("MAX_VOC"),
                min(instalacoes.VOC).alias("MIN_VOC"),
                sum(instalacoes.NOx).alias("SUM_NOx"),
                avg(instalacoes.NOx).alias("AVG_NOx"),
                max(instalacoes.NOx).alias("MAX_NOx"),
                min(instalacoes.NOx).alias("MIN_NOx"),
                sum(instalacoes.CO).alias("SUM_CO"),
                avg(instalacoes.CO).alias("AVG_CO"),
                max(instalacoes.CO).alias("MAX_CO"),
                min(instalacoes.CO).alias("MIN_CO"),
                sum(instalacoes.CO2).alias("SUM_CO2"),
                avg(instalacoes.CO2).alias("AVG_CO2"),
                max(instalacoes.CO2).alias("MAX_CO2"),
                min(instalacoes.CO2).alias("MIN_CO2"),
                sum(instalacoes.Particulates).alias("SUM_Particulates"),
                avg(instalacoes.Particulates).alias("AVG_Particulates"),
                max(instalacoes.Particulates).alias("MAX_Particulates"),
                min(instalacoes.Particulates).alias("MIN_Particulates"),
                sum(instalacoes.PM10).alias("SUM_PM10"),
                avg(instalacoes.PM10).alias("AVG_PM10"),
                max(instalacoes.PM10).alias("MAX_PM10"),
                min(instalacoes.PM10).alias("MIN_PM10"),
                sum(instalacoes.PM2_5).alias("SUM_PM2_5"),
                avg(instalacoes.PM2_5).alias("AVG_PM2_5"),
                max(instalacoes.PM2_5).alias("MAX_PM2_5"),
                min(instalacoes.PM2_5).alias("MIN_PM2_5"),
                sum(instalacoes.HAPS).alias("SUM_HAPS"),
                avg(instalacoes.HAPS).alias("AVG_HAPS"),
                max(instalacoes.HAPS).alias("MAX_HAPS"),
                min(instalacoes.HAPS).alias("MIN_HAPS"),
                sum(instalacoes.SO2).alias("SUM_SO2"),
                avg(instalacoes.SO2).alias("AVG_SO2"),
                max(instalacoes.SO2).alias("MAX_SO2"),
                min(instalacoes.SO2).alias("MIN_SO2")
               )

In [7]:
instalacoes.toPandas()

Year        Borough                         Facility_Name    latitude  \
0    2011         Queens             CON ED - ASTORIA FACILITY  40.759776    
1    2011          Bronx       AMALGAMATED HOUSING-130 GALE PL  40.826153    
2    2011       Brooklyn                    N 1ST STREET PLANT  40.692529    
3    2010         Queens  NATIONAL GRID - FAR ROCKAWAY STATION  40.759776    
4    2011  Staten Island        ARTHUR KILL GENERATING STATION  40.643498    
..    ...            ...                                   ...         ...   
950  2019      Manhattan                  MOUNT SINAI HOSPITAL  40.713054    
951  2019       Brooklyn            NAP - KENT AVENUE FACILITY  40.692529    
952  2019       Brooklyn                 GREENPOINT ENERGY CTR  40.692529    
953  2019         Queens                     VERNON BLVD PLANT  40.759776    
954  2019          Bronx         PARKCHESTER SOUTH CONDOMINIUM  40.826153    

       longitude  SUM_VOC  AVG_VOC  MAX_VOC  MIN_VOC  SUM_NOx  ...  MAX_PM2_5  \
0     -73.817302     5.56     5.56     5.56     5.56     2.47  ...       0.15   
1     -73.920265     0.34     0.34     0.34     0.34     8.25  ...       0.46   
2     -73.990996     0.98     0.98     0.98     0.98     4.58  ...       2.04   
3     -73.817302     5.71     5.71     5.71     5.71    78.42  ...       7.94   
4     -74.076203    48.03    48.03    48.03    48.03   772.09  ...      66.00   
..           ...      ...      ...      ...      ...      ...  ...        ...   
950   -74.007228     1.31     1.31     1.31     1.31    20.26  ...       1.74   
951   -73.990996    54.06    54.06    54.06    54.06     0.38  ...       0.00   
952   -73.990996     1.38     1.38     1.38     1.38     5.56  ...       0.08   
953   -73.817302     0.42     0.42     0.42     0.42     3.16  ...       1.20   
954   -73.920265     2.18     2.18     2.18     2.18    69.71  ...       2.76   

     MIN_PM2_5  SUM_HAPS  AVG_HAPS  MAX_HAPS  MIN_HAPS  SUM_SO2  AVG_SO2  \
0         0.15       0.0       0.0       0.0       0.0     0.02     0.02   
1         0.46       0.0       0.0       0.0       0.0     1.39     1.39   
2         2.04       0.0       0.0       0.0       0.0     0.28     0.28   
3         7.94       0.0       0.0       0.0       0.0     0.65     0.65   
4        66.00       0.0       0.0       0.0       0.0     5.29     5.29   
..         ...       ...       ...       ...       ...      ...      ...   
950       1.74       0.0       0.0       0.0       0.0     0.44     0.44   
951       0.00       0.0       0.0       0.0       0.0     0.00     0.00   
952       0.08       0.0       0.0       0.0       0.0     0.00     0.00   
953       1.20       0.0       0.0       0.0       0.0     0.13     0.13   
954       2.76       0.0       0.0       0.0       0.0    11.62    11.62   

     MAX_SO2  MIN_SO2  
0       0.02     0.02  
1       1.39     1.39  
2       0.28     0.28  
3       0.65     0.65  
4       5.29     5.29  
..       ...      ...  
950     0.44     0.44  
951     0.00     0.00  
952     0.00     0.00  
953     0.13     0.13  
954    11.62    11.62  

[955 rows x 41 columns]

In [8]:
# write to delta table
instalacoes \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/InstalacoesCounty")

In [9]:
spark.table("PL7G1_GOLD.InstalacoesCounty").toPandas()

Year        Borough                         Facility_Name  SUM_VOC  \
0    2011         Queens             CON ED - ASTORIA FACILITY     5.56   
1    2011          Bronx       AMALGAMATED HOUSING-130 GALE PL     0.34   
2    2011       Brooklyn                    N 1ST STREET PLANT     0.98   
3    2010         Queens  NATIONAL GRID - FAR ROCKAWAY STATION     5.71   
4    2011  Staten Island        ARTHUR KILL GENERATING STATION    48.03   
..    ...            ...                                   ...      ...   
950  2019      Manhattan                  MOUNT SINAI HOSPITAL     1.31   
951  2019       Brooklyn            NAP - KENT AVENUE FACILITY    54.06   
952  2019       Brooklyn                 GREENPOINT ENERGY CTR     1.38   
953  2019         Queens                     VERNON BLVD PLANT     0.42   
954  2019          Bronx         PARKCHESTER SOUTH CONDOMINIUM     2.18   

     AVG_VOC  MAX_VOC  MIN_VOC  SUM_NOx  AVG_NOx  MAX_NOx  ...  SUM_HAPS  \
0       5.56     5.56     5.56     2.47     2.47     2.47  ...       0.0   
1       0.34     0.34     0.34     8.25     8.25     8.25  ...       0.0   
2       0.98     0.98     0.98     4.58     4.58     4.58  ...       0.0   
3       5.71     5.71     5.71    78.42    78.42    78.42  ...       0.0   
4      48.03    48.03    48.03   772.09   772.09   772.09  ...       0.0   
..       ...      ...      ...      ...      ...      ...  ...       ...   
950     1.31     1.31     1.31    20.26    20.26    20.26  ...       0.0   
951    54.06    54.06    54.06     0.38     0.38     0.38  ...       0.0   
952     1.38     1.38     1.38     5.56     5.56     5.56  ...       0.0   
953     0.42     0.42     0.42     3.16     3.16     3.16  ...       0.0   
954     2.18     2.18     2.18    69.71    69.71    69.71  ...       0.0   

     AVG_HAPS  MAX_HAPS  MIN_HAPS  SUM_SO2  AVG_SO2  MAX_SO2  MIN_SO2  \
0         0.0       0.0       0.0     0.02     0.02     0.02     0.02   
1         0.0       0.0       0.0     1.39     1.39     1.39     1.39   
2         0.0       0.0       0.0     0.28     0.28     0.28     0.28   
3         0.0       0.0       0.0     0.65     0.65     0.65     0.65   
4         0.0       0.0       0.0     5.29     5.29     5.29     5.29   
..        ...       ...       ...      ...      ...      ...      ...   
950       0.0       0.0       0.0     0.44     0.44     0.44     0.44   
951       0.0       0.0       0.0     0.00     0.00     0.00     0.00   
952       0.0       0.0       0.0     0.00     0.00     0.00     0.00   
953       0.0       0.0       0.0     0.13     0.13     0.13     0.13   
954       0.0       0.0       0.0    11.62    11.62    11.62    11.62   

       longitude    latitude  
0     -73.817302  40.759776   
1     -73.920265  40.826153   
2     -73.990996  40.692529   
3     -73.817302  40.759776   
4     -74.076203  40.643498   
..           ...         ...  
950   -74.007228  40.713054   
951   -73.990996  40.692529   
952   -73.990996  40.692529   
953   -73.817302  40.759776   
954   -73.920265  40.826153   

[955 rows x 41 columns]

In [10]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/InstalacoesCounty/`
""").show()

++
||
++
++



In [12]:
spark.sql("""
DROP TABLE IF EXISTS PL7G1_GOLD.InstalacoesCounty_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.InstalacoesCounty_presto (
        Year INT,
        Borough VARCHAR(50),
        Facility_Name VARCHAR(100),
        SUM_VOC Double,
        AVG_VOC Double,
        MAX_VOC Double,
        MIN_VOC Double,
        SUM_NOx Double,
        AVG_NOx Double,
        MAX_NOx Double,
        MIN_NOx Double,
        SUM_CO Double,
        AVG_CO Double,
        MAX_CO Double,
        MIN_CO Double,
        SUM_CO2 Double,
        AVG_CO2 Double,
        MAX_CO2 Double,
        MIN_CO2 Double,
        SUM_Particulates Double,
        AVG_Particulates Double,
        MAX_Particulates Double,
        MIN_Particulates Double,
        SUM_PM10 Double,
        AVG_PM10 Double,
        MAX_PM10 Double,
        MIN_PM10 Double,
        SUM_PM2_5 Double,
        AVG_PM2_5 Double,
        MAX_PM2_5 Double,
        MIN_PM2_5 Double,
        SUM_HAPS Double,
        AVG_HAPS Double,
        MAX_HAPS Double,
        MIN_HAPS Double,
        SUM_SO2 Double,
        AVG_SO2 Double,
        MAX_SO2 Double,
        MIN_SO2 Double,
        longitude VARCHAR(20),
       latitude VARCHAR(20)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/InstalacoesCounty/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

